# Model Selection

- Model bizim ML algoritmalarimiza deniyor, mesela KNN.
- Model selection ise en iyi parametreleri seçmek anlamina gelir.
- parametreler:
- 1) Ogrenilen parametreler: biz secemiyoruz model kendi ogreniyor.
- 2) Secilen parametreler: bunlara hyperparameter denir. Bunlari biz seceriz örnegin KNN algoritmasindaki K degeri.
- ML alogritmasini nasil degerlendirecegimizi belirlemeliyiz. Accuracy mi? Confusion matrix mi? gibi.
- Sectigimiz modeller sonrasi ortaya cikan accuracy gibi degerlerin tutarli olmasi gerekir. Mesela bir model ve test datamiz var accuracy %89 cikti, ayni modeli baska bir test datasiyla denedik sonuc %60 cikti. Bu problrmi bir test datasiyla denedik sonuc %60 cikti. Bu problemi cozebilmemiz icin K-fold cross validation diye bir yontem kullanabiliriz.

# K-Fold Cross Validation

- avoid overfitting: k-fold cross validation overfitting onleme yontemlerindendir.
- k=3 icin
- once datamizi train ve test olarak ayiriyoruz.
- TRAIN | TEST
- daha sonra test kismini unutuyoruz. Train kismini k degeri kadar boluyoruz.
- TRAIN | TRAIN | VALIDATION
- TRAIN | VALIDATION | TRAIN
- VALIDATION | TRAIN | TRAIN
- Datamizi trainlerle train edip validation ile dogrulama yapiyoruz.
- K degeri kadar accuracy elde edip ortalama accuracy'i hesapliyoruz.
- Daha sonra test datasiyla test ediyoruz.

In [2]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np 

In [3]:
iris = load_iris()

In [50]:
x= iris.data
y = iris.target 

In [51]:
#knn algoritmasi kullanacagimiz icin normalizasyon yapiyoruz.
x = (x-np.min(x))/(np.max(x)-np.min(x))

In [52]:
#train test split 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3)

In [53]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3) #k degeri

In [54]:
# K - Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = knn , X= x_train, y=y_train, cv=10)

- estimators = knn demek cross validation yaparken knn algoritmasini kullan demek.
- K fold degeri icin 10 olmasini istiyorum ve bunun icin cv = 10 yaziyorum. Bu ne demek: train datasini 10'a boldum her seferinde 9 tanesi train bir tanesini ise validation olarak kullaniyorum ve bunu 10 kez yapıyorum ve her seferinde bir tane accuracy degeri elde ediyorum.
- Accuracy degerlerine bakalim : 

In [15]:
print(accuracies)

[1.         0.90909091 1.         1.         1.         0.9
 1.         0.8        1.         0.9       ]


- Simdi ortalamsina bakmaliyiz: 

In [17]:
print("avarege accuray: ",np.mean(accuracies))

avarege accuray:  0.9509090909090909


In [19]:
print("avarege accuray: ",np.std(accuracies)) #datanin tutarli olmasi

avarege accuray:  0.0664569704818075


In [20]:
knn.fit(x_train,y_train)
print("test accuracy: ",knn.score(x_test,y_test))

test accuracy:  0.9555555555555556


#  Grid Search Cross Validation 

- Belirlenen k degerleri verilen cv degeri kadar validation yapilarak best accuracy'e sahip k degeri bulunur(knn icin k degeri).

In [71]:
from sklearn.model_selection import GridSearchCV

In [72]:
grid = {"n_neighbors":np.arange(1,50)} #knn'deki k 1'den 50'ye kadar denenecek
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(estimator=knn,param_grid=grid,cv= 10) #GridSearchCV cross val'deki validation 10 kez olacak.
knn_cv.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])})

In [73]:
print("tuned hyperparameter k: ", knn_cv.best_params_)
print("tuned parametreye gore en iyi accuracy(best score):", knn_cv.best_score_)

tuned hyperparameter k:  {'n_neighbors': 7}
tuned parametreye gore en iyi accuracy(best score): 0.99


- yani en iyi accuracy knn algoritmasında k degeri 13 oldugunda %98 olacakmis


# Grid Search CV with Logistic Regression

In [57]:
x = x[:100,:] #ilk 100 degeri x'e esitle
y =y[:100] # ilk 100 degeri al

In [59]:
from  sklearn.linear_model import LogisticRegression

In [60]:
grid = {"C":np.logspace(-3,3,7),"penalty":["l1","l2"]} 

- c buyukse overfit olur, c kucukse underfit olur.
- l1 ve l2 regularization icin kullanilan loss functionlaridir.

In [67]:
logreg = LogisticRegression()
logreg_cv = GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])})

In [68]:
print("tuned hyperparameter : ", logreg_cv.best_params_)

tuned hyperparameter :  {'C': 100.0}


In [70]:
print("tuned parametreye gore en iyi accuracy(best score):", logreg_cv.best_score_)

tuned parametreye gore en iyi accuracy(best score): 0.9800000000000001
